In [12]:
import copy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import load_data_template
from src.features.games_detailed import GameDetailedFeatures

In [147]:
import importlib
importlib.reload(src.features.games_detailed)
from src.features.games_detailed import GameDetailedFeatures
data = load_data_template(season=False)
game_detail_feat = GameDetailedFeatures(default_lags=3)
data = game_detail_feat.per_team_wrapper(data, game_detail_feat.detail_features_by_game, per_day=True)
data.dropna(inplace=True)
print(data.shape)
data.sample()

(981, 85)


Season team_a team_b  in_target  game_set a_win DayNum  \
10586   2008   1139   1375      False         0  True    137   

       FGM_game_team_a_lag-1  FGM_game_team_a_lag-2  FGM_game_team_a_lag-3  \
10586                   19.0                   22.0                   24.0   

               ...           TO_game_team_b_lag-3  Stl_game_team_b_lag-1  \
10586          ...                           19.0                    5.0   

       Stl_game_team_b_lag-2  Stl_game_team_b_lag-3  Blk_game_team_b_lag-1  \
10586                    5.0                    6.0                    0.0   

       Blk_game_team_b_lag-2  Blk_game_team_b_lag-3  PF_game_team_b_lag-1  \
10586                    2.0                    5.0                  24.0   

       PF_game_team_b_lag-2  PF_game_team_b_lag-3  
10586                  21.0                  21.0  

[1 rows x 85 columns]

In [148]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss


cat_cols = ['team_a', 'team_b', 'game_set']
X = data.drop(['Season', 'a_win', 'in_target', 'DayNum'], axis=1)
X_enc = OneHotEncoder(categorical_features=[c in cat_cols for c in X.columns]).fit_transform(X).todense()
y = data[['a_win', 'game_set']].astype({'a_win': int})

kf = KFold(n_splits=3, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X_enc):
    X_tr, y_tr = X_enc[tr_i], y.iloc[tr_i].a_win
    X_t, y_t = X_enc[t_i], y.iloc[t_i].a_win
    xgb = XGBClassifier(nthreads=4)
    xgb.fit(X_tr, y_tr)
    preds = xgb.predict_proba(X_t)
    ncaa_true = y.iloc[t_i][y.iloc[t_i].game_set == 0]
    ncaa_pred = preds[y.iloc[t_i].reset_index().game_set == 0]
    metric = log_loss(ncaa_true.a_win.astype(int), ncaa_pred)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

0.1825622310228247
0.1890674890182764
0.208754024702261
Metric Mean: 0.19 (0.01)


In [149]:
from xgboost import plot_importance

plt.figure(figsize=(20,4))
plot_importance(xgb, max_num_features=15)
plt.show()